<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_application/langchain_output_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LangChain Explained In 15 Minutes - A MUST Learn For Python Programmers](https://www.youtube.com/watch?v=mrjq3lFz23s)

test

In [ ]:
# packages:
%%capture --no-stderr
%pip install -U langgraph langsmith langchain_google_genai langchain   python-dotenv

In [ ]:
# API keys set up
# Configure environment to connect to LangSmith.
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain_applications"

In [ ]:
import os
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

[Output parser](https://python.langchain.com/docs/concepts/output_parsers/)

**Output parser** is responsible for taking the output of a model and transforming it to a more suitable format for downstream tasks. Useful when you are using LLMs to generate structured data, or to normalize output from chat models and LLMs.

**`ChatPromptTemplate.from_messages()`**

**`llm.predict_messages()`**

#### Language Translation Agent with LangChain

**`ChatPromptTemplate.from_messages()`**

**`llm.predict_messages()`**

In [ ]:
# Language Translation Agent with LangChain

# Import Required Libraries
from langchain_core.prompts import ChatPromptTemplate

# Initialize Language Model
chat_model = llm

# Define Prompt Templates
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

# Create a prompt template
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template)
    ]
)

# Format the messages using the template
messages = chat_prompt.format_messages(
    input_language="English",
    output_language="French",
    text="I am an expert in AI Agents."
)

# chat_model = llm
# Use the chat model to predict the response (notice)
result = chat_model.predict_messages(messages)

# Output the translated content
print(result.content)


<ipython-input-8-e8e4d9dc0a65>:25: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat_model.predict_messages(messages)


Je suis expert en agents IA.



In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Assuming llm is already defined
chat_model = llm

template = """You are a helpful assistant that solves math problems and shows your work.
Output each step and then return the answer in the following format:
answer = <answer here>.
Make sure to output the answer in all lowercase with exactly one space and one equal sign following it'answer'.
"""

human_template = "{problem}"

# Correctly initializing the ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)

# Formatting the messages
messages = chat_prompt.format_messages(problem="2x^2 - 5x + 3 = 0")

# Getting the response
result = llm.predict_messages(messages)

# Direct parsing of the content
response_content = result.content

# Print the response for debugging
print("Response from LLM:", response_content)

# Parsing steps (optional, based on how the output is structured)
# Here, parsing should handle splitting steps and the final answer
if "answer =" in response_content:
    steps, answer = response_content.rsplit("answer =", 1)
    print("Steps:\n", steps.strip())
    print("Answer:", answer.strip())
else:
    print("Could not parse the answer.")


Response from LLM: Here's how to solve the quadratic equation 2x² - 5x + 3 = 0:

**1. Factoring**

We look for two numbers that add up to -5 (the coefficient of x) and multiply to 6 (the product of the coefficient of x² and the constant term). Those numbers are -2 and -3.  We can rewrite the equation as:

2x² - 2x - 3x + 3 = 0

**2. Grouping**

Now we group the terms and factor:

2x(x - 1) - 3(x - 1) = 0

**3. Factoring out the common term**

(x - 1)(2x - 3) = 0

**4. Solving for x**

This equation is true if either (x - 1) = 0 or (2x - 3) = 0.  Solving each gives us:

x - 1 = 0  =>  x = 1

2x - 3 = 0  =>  2x = 3  =>  x = 3/2

**Answer = 1 and 3/2**

Could not parse the answer.


#### **Output Parse**

In [ ]:
# Output Parse
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import output_parser, BaseOutputParser

class AnswerOutputParser(BaseOutputParser):
  def parse(self, text: str):
    """Parse the output of an LLM call. """
    return text.strip().split("answer = ")


chat_model = llm

template = """ You are a helpful assistant that solves math problems and show your work.
           output each step then return the answer in the following format : answer = <answer here>.
           Make sure to output answer in all lowercase and to have exactly one space and one equal sign follow it.
           """

human_template = "{problem}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)

messages = chat_prompt.format_messages(problem="2x2 - 5x + 3 = 0")

result = llm.predict_messages(messages)

parsed = AnswerOutputParser().parse(result.content)
steps, answer = parsed

print(answer)

x = 1, x = 3/2


In [ ]:
# Comma Seprated List Output Parse in Chain

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import output_parser, BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
  def parse(self, text: str):
    return text.strip().split(", ")


chat_model = llm

template = """ You are a helpful assistant who gerates comma separated lists.
           A user will pass in a category, and you should generate 5 objects in that category in a comma seprated List.
           ONLY return a comma separated list, nothing more.
           """

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)

# | use to combine all of them into one object
chain = chat_prompt | llm | CommaSeparatedListOutputParser()

result = chain.invoke({"text": "colors"})

print(result)


['red', 'green', 'blue', 'yellow', 'orange']
